In [2]:
!python -m spacy download en_core_web_sm
!pip install geotext
!pip install spacy

2023-03-03 09:53:48.768286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 09:53:49.431674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-03 09:53:49.431732: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-03 09:53:49.431741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [27]:
import spacy
import random
from geotext import GeoText
nlp = spacy.load('en_core_web_sm')
  
sentence = "Apple is looking at buying U.K. startup for $1 billion"
  
doc = nlp(sentence)
  
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [2]:
path_data_travel_questions = '5000TravelQuestionsDataset.csv'

In [3]:
import csv

with open(path_data_travel_questions, newline='',encoding='latin-1') as f:
    reader = csv.reader(f)
    data = list(reader)
data_travel_questions = [i[0] for i in data]

In [39]:
content_string = "\n".join(data_travel_questions)

In [4]:
allow_ner_types = ['PERSON', 'PRODUCT', 'WORK_OF_ART', 'EVENT', 'FAC', 'GPE', 
                   'LANGUAGE', 'LAW', 'LOC', 'NORP', 'ORG', 'PERSON', 'PRODUCT']

In [5]:
rwandan_cities = ['Kigali','Gisenyi','Ruhengeri','Butare','Muhanga','Byumba','Cyangugu','Nyanza','Bugarama','Kayonza','Rwamagana','Nyamata','Ruhango','Gikongoro','Nyagatare','Busogo','Kibuye','Kibungo','Rubengera']
rivers = ["Rusizi River","Sebeya River","Koko River","Rubyiro River","Ruhwa River","Akagera River","Nyabarongo River","Mukungwa River","Akanyaru River","Rukara River","Mbirurume River"]
lakes = ["Lake Kivu","Lake Ihema","Lake Rweru","Lake Rwanyakizinga","Lake Hago","Lake Mpanga","Lake Ruhondo","Lake Muhazi","Lake Cyohoha","Lake Kivumba","Lake Burera","Lake Mugesera","Lake Sake","Lake Mihindi","Lake Mpanga"]
mountains = ["Mount Karisimbi","Mount Gahinga","Mount Sabyinyo","Mount Bisoke","Mount Huye","Mount Nyiragongo","Mount Muhabura"]
parks_and_forests = ["Akagera National Park","Volcanoes National Park","Bwindi Impenetrable Forest","Nyungwe National Park","Cyamudongo Forest"]
islands = ["Bushongo Island","Cyuza Island","Idwi Island","Ihoka Island","Ika Island","Nkombo Island","Munanira Island","Ishovu Island","Ite Island","Kwankoro Island","Sharita Island","Gombo Island","Iwawa Island","Iwinja Island","Kigwa Island","Kikaya Island","Mishungo Island","Bugarura Island"]


In [22]:
cities = {}
countries = set()
other_places = set()
for sentence in data_travel_questions:
    doc = nlp(sentence)
    for ent in doc.ents:
        if ent.label_ in allow_ner_types:
            if ent.label_ == 'GPE':
                place = GeoText(ent.text)
                if place.countries:
                    countries.add(ent.text)
                elif place.cities:
                    cities[ent.text] = cities.get(ent.text,0) + 1
            if ent.label_ == 'LOC':
                other_places.add(ent.text)


In [29]:
other_places_map = {}
for place in other_places:
    if "island" in place or "isle" in place:
        other_places_map[place] = random.choice(islands)
    elif "Mount" in place or "Mt." in place:
        other_places_map[place] = random.choice(mountains)
    elif "River" in place:
        other_places_map[place] = random.choice(rivers)
    elif "Park" in place:
        other_places_map[place] = random.choice(parks_and_forests)
    elif "Lake" in place:
        other_places_map[place] = random.choice(lakes)

In [31]:
cities_mapping = {}
for city,frequency in cities.items():
    if frequency >= 10:
        cities_mapping[city] = "Kigali"
    else:
        cities_mapping[city] = random.choice(rwandan_cities)

In [40]:
for country in countries:
    content_string = content_string.replace(country,"Rwanda")
for original_city,substitute_city in cities_mapping.items():
    content_string = content_string.replace(original_city,substitute_city)
for original_place,substitute_place in other_places_map.items():
    content_string = content_string.replace(original_place,substitute_place)

In [42]:
substituted_data_travel_content = content_string.split("\n")

In [45]:
substituted_data_travel_content

['What are the special things we (husband and me) can do during a 5 day stay at Nyanza?',
 'What are the companies which organize shark feeding events for scuba divers?',
 'Is it safe for female traveller to go alone to Nyanza?',
 'What are the best places around Nyanza for safari?',
 'What are the best places to stay for a family to stay away from nightlife?',
 'What are the train services that travels from cape town to Oudtshoorn?',
 'What are the best places to spend about 2 weeks relaxing honeymoon in Rwanda?',
 'Can we use travellers cheques and credit cards in Nyanza?',
 'Is it warm enough to swim in early September in Nyanza?',
 'What are the best beaches for shelling in CapeTown?',
 'I want to find a central location in Nyanza to stay with my family with ease access of food and necessities?',
 'What are the differences between Shamwari and Kwandwe game reserves and what is the best?',
 'From where can I find adventure activity camps in Rwanda?',
 'How can I select an apartment 

In [46]:
with open("5000_travel_sentences_data_substituted.txt","w") as f:
    # for line in substituted_data_travel_content:
    f.write(substituted_data_travel_content,"\n")